To do an import you need a "stable identifier" as the error message says. This means that you need to have a val, not a var. Since you defined spark as a var, scala can't import correctly.

In [4]:
val sc = spark
import sc.implicits._

sc = org.apache.spark.sql.SparkSession@2963a0cb


org.apache.spark.sql.SparkSession@2963a0cb

In [1]:
case class Flight(DEST_COUNTRY_NAME: String,
                  ORIGIN_COUNTRY_NAME: String,
                  count: BigInt)

defined class Flight


In [5]:
val flightsDF = sc.read
  .parquet("/home/jovyan/work/Spark-The-Definitive-Guide-master/data/flight-data/parquet/2010-summary.parquet/")

flightsDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [6]:
val flights = flightsDF.as[Flight]

flights = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [11]:
flights
  .filter(flight_row => flight_row.ORIGIN_COUNTRY_NAME != "Canada")
  .map(flight_row => flight_row)
  .take(5)

Array(Flight(United States,Romania,1), Flight(United States,Ireland,264), Flight(United States,India,69), Flight(Egypt,United States,24), Flight(Equatorial Guinea,United States,1))

In [8]:
flights
  .take(5)
  .filter(flight_row => flight_row.ORIGIN_COUNTRY_NAME != "Canada")
  .map(fr => Flight(fr.DEST_COUNTRY_NAME, fr.ORIGIN_COUNTRY_NAME, fr.count + 5))


Array(Flight(United States,Romania,6), Flight(United States,Ireland,269), Flight(United States,India,74), Flight(Egypt,United States,29), Flight(Equatorial Guinea,United States,6))

<h3>Structured Streamig</h3>

In [13]:
val staticDataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("/home/jovyan/work/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")

staticDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


lastException: Throwable = null


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [14]:
staticDataFrame.createOrReplaceTempView("retail_data")
val staticSchema = staticDataFrame.schema

staticSchema = StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))


StructType(StructField(InvoiceNo,StringType,true), StructField(StockCode,StringType,true), StructField(Description,StringType,true), StructField(Quantity,IntegerType,true), StructField(InvoiceDate,TimestampType,true), StructField(UnitPrice,DoubleType,true), StructField(CustomerID,DoubleType,true), StructField(Country,StringType,true))

In [15]:
import org.apache.spark.sql.functions.{window, column, desc, col}
staticDataFrame
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))
  .sum("total_cost")
  .show(5)

+----------+--------------------+-----------------+
|CustomerId|              window|  sum(total_cost)|
+----------+--------------------+-----------------+
|   16057.0|[2011-12-05 00:00...|            -37.6|
|   14126.0|[2011-11-29 00:00...|643.6300000000001|
|   13500.0|[2011-11-16 00:00...|497.9700000000001|
|   17160.0|[2011-11-08 00:00...|516.8499999999999|
|   15608.0|[2011-11-11 00:00...|            122.4|
+----------+--------------------+-----------------+
only showing top 5 rows



In [16]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [17]:
val streamingDataFrame = spark.readStream
    .schema(staticSchema)
    .option("maxFilesPerTrigger", 1)
    .format("csv")
    .option("header", "true")
    .load("/home/jovyan/work/Spark-The-Definitive-Guide-master/data/retail-data/by-day/*.csv")

streamingDataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]


[InvoiceNo: string, StockCode: string ... 6 more fields]

In [18]:
streamingDataFrame.isStreaming // returns true

true

In [19]:
val purchaseByCustomerPerHour = streamingDataFrame
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")
  .groupBy(
    $"CustomerId", window($"InvoiceDate", "1 day"))
  .sum("total_cost")


purchaseByCustomerPerHour = [CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]


[CustomerId: double, window: struct<start: timestamp, end: timestamp> ... 1 more field]

In [20]:
purchaseByCustomerPerHour.writeStream
    .format("memory") // memory = store in-memory table
    .queryName("customer_purchases") // the name of the in-memory table
    .outputMode("complete") // complete = all the counts should be in the table
    .start()


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2a1365a0

In [21]:
spark.sql("""
  SELECT *
  FROM customer_purchases
  ORDER BY `sum(total_cost)` DESC
  """)
  .show(5)

+----------+------+---------------+
|CustomerId|window|sum(total_cost)|
+----------+------+---------------+
+----------+------+---------------+



<h3>Machine Learning</h3>

In [22]:
staticDataFrame.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [23]:
import org.apache.spark.sql.functions.date_format
val preppedDataFrame = staticDataFrame
  .na.fill(0)
  .withColumn("day_of_week", date_format($"InvoiceDate", "EEEE"))
  .coalesce(5)

preppedDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [24]:
val trainDataFrame = preppedDataFrame
  .where("InvoiceDate < '2011-07-01'")
val testDataFrame = preppedDataFrame
  .where("InvoiceDate >= '2011-07-01'")

trainDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]
testDataFrame = [InvoiceNo: string, StockCode: string ... 7 more fields]


[InvoiceNo: string, StockCode: string ... 7 more fields]

In [25]:
trainDataFrame.count()
testDataFrame.count()

296006

In [26]:
import org.apache.spark.ml.feature.StringIndexer
val indexer = new StringIndexer()
  .setInputCol("day_of_week")
  .setOutputCol("day_of_week_index")

indexer = strIdx_f3c494a412f2


strIdx_f3c494a412f2

In [27]:
import org.apache.spark.ml.feature.OneHotEncoder
val encoder = new OneHotEncoder()
  .setInputCol("day_of_week_index")
  .setOutputCol("day_of_week_encoded")

encoder = oneHot_f1e5da708c4f


oneHot_f1e5da708c4f

In [28]:
import org.apache.spark.ml.feature.VectorAssembler

val vectorAssembler = new VectorAssembler()
  .setInputCols(Array("UnitPrice", "Quantity", "day_of_week_encoded"))
  .setOutputCol("features")

vectorAssembler = vecAssembler_8597d61e5176


vecAssembler_8597d61e5176

In [30]:
import org.apache.spark.ml.Pipeline

val transformationPipeline = new Pipeline()
  .setStages(Array(indexer, encoder, vectorAssembler))

transformationPipeline = pipeline_4015fb929137


pipeline_4015fb929137

In [31]:
val fittedPipeline = transformationPipeline.fit(trainDataFrame)

fittedPipeline = pipeline_4015fb929137


pipeline_4015fb929137

In [32]:
val transformedTraining = fittedPipeline.transform(trainDataFrame)


transformedTraining = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [35]:
transformedTraining.cache()

[InvoiceNo: string, StockCode: string ... 10 more fields]

In [34]:
import org.apache.spark.ml.clustering.KMeans
val kmeans = new KMeans()
  .setK(20)
  .setSeed(1L)

kmeans = kmeans_674b7e3a07c3


kmeans_674b7e3a07c3

In [36]:
import org.apache.spark.ml.clustering.KMeans
val kmeans = new KMeans()
  .setK(20)
  .setSeed(1L)

kmeans = kmeans_25199358aab7


kmeans_25199358aab7

In [37]:
val kmModel = kmeans.fit(transformedTraining)

kmModel = kmeans_25199358aab7


kmeans_25199358aab7

In [38]:
kmModel.computeCost(transformedTraining)

8.455373996537484E7

In [39]:
val transformedTest = fittedPipeline.transform(testDataFrame)

transformedTest = [InvoiceNo: string, StockCode: string ... 10 more fields]


[InvoiceNo: string, StockCode: string ... 10 more fields]

In [40]:
kmModel.computeCost(transformedTest)


5.175070947222117E8

<h3>低階API</h3>

In [43]:
spark.sparkContext.parallelize(Seq(1, 2, 3)).toDF()

[value: int]